This first example of the basic usage of the Pyriod package was demonstrated at the [TASC5/KASC12 workshop](https://web.mit.edu/tasc5/) in Boston 22-26 July 2019.  It shows how to reproduce the frequency analysis of a pulsating helium-atmosphere white dwarf presented in [Bell et al. (A&A, 632, A42)](https://ui.adsabs.harvard.edu/abs/2019A%26A...632A..42B).  

See the online [tutorial](https://pyriod.readthedocs.io/en/latest/tutorial/) for more info about how to use Pyriod.

In [1]:
%matplotlib widget
import lightkurve as lk
from Pyriod import Pyriod
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dat = np.loadtxt('/Users/keatonb/Research/WDJ1106+1802/wqed_lcs/all.lc')
time = dat[:,0]/(24.*3600.) #in days
flux = dat[:,1]
april = np.where((time > 66) & (time < 70))

In [4]:
#Download light curve of a DBV white dwarf pulsator observed by TESS
#lc = lk.search_lightcurvefile('TIC 257459955',mission='TESS',sector=3).download().get_lightcurve("PDCSAP_FLUX").flatten(2161)
#lc = lk.LightCurve(1001+np.arange(400),1+np.sin(np.arange(400))+.1*np.random.randn(400))
#lc = lk.search_lightcurvefile('TIC 239678779',mission='TESS',sector=19).download().get_lightcurve("PDCSAP_FLUX").flatten(2161)
#lc = lc.remove_nans()

In [3]:
#Instantiate Pyriod object
#pyriod = Pyriod(lc)
pyriod = Pyriod(time=time[april],flux=flux[april],nyquist_factor=0.5)

In [4]:
#Interact with time series
pyriod.TimeSeries()

str

In [7]:
tab = pyriod.per_orig.to_table()

In [8]:
np.savetxt('lightcurve.txt',np.vstack((pyriod.lc_orig.time,pyriod.lc_orig.flux)).T)

In [9]:
np.savetxt('periodogram.txt',np.vstack((pyriod.per_orig.frequency.value,pyriod.per_orig.power.value)).T)


In [5]:
#Interact with periodogram
pyriod.Periodogram()

In [6]:
#Interact with frequency solution
pyriod.Signals()

In [7]:
#Interact with log
pyriod.Log()

In [12]:
def sigthreshold(snr = 5., fmin=None, fmax=None, winspacing = 100, winwidth = 100):
    if fmin is None:
        fmin = np.min(pyriod.freqs)
    if fmax is None:
        fmax = np.max(pyriod.freqs)
    wincenters = np.arange(fmin + winwidth/2.,fmax - winwidth/2.,winspacing)
    means = np.zeros(wincenters.shape)
    for i in range(len(wincenters)):
        inwin = np.where((pyriod.freqs >= wincenters[i] - winwidth/2.) & (pyriod.freqs < wincenters[i] + winwidth/2.))[0]
        if len(inwin) > 0:
            means[i] = np.mean(pyriod.per_resid[inwin].power.value)
        else:
            means[i] = np.nan
    return wincenters,means*snr
    

In [13]:
pyriod.perax.plot(sigthreshold()[0],sigthreshold()[1],ls='--',c='r')

In [14]:
pyriod.perfig.canvas.draw()

In [48]:
sigthreshold()

(array([  50.03989875,  150.03989875,  250.03989875,  350.03989875,
         450.03989875,  550.03989875,  650.03989875,  750.03989875,
         850.03989875,  950.03989875, 1050.03989875, 1150.03989875,
        1250.03989875, 1350.03989875, 1450.03989875, 1550.03989875,
        1650.03989875, 1750.03989875, 1850.03989875, 1950.03989875,
        2050.03989875, 2150.03989875, 2250.03989875, 2350.03989875,
        2450.03989875, 2550.03989875, 2650.03989875, 2750.03989875,
        2850.03989875, 2950.03989875, 3050.03989875, 3150.03989875,
        3250.03989875, 3350.03989875, 3450.03989875, 3550.03989875,
        3650.03989875, 3750.03989875, 3850.03989875, 3950.03989875,
        4050.03989875]),
 array([28.32273923, 23.97007747, 19.25694495, 17.9035875 , 16.97457346,
        17.98858551, 17.67309004, 17.4528343 , 16.76670236, 15.69426128,
        16.73878622, 15.93269047, 17.20388103, 16.28612921, 17.3003272 ,
        16.79179852, 16.36991363, 15.26384776, 18.3224088 , 17.14840052,
   

In [7]:
pyriod.values

,include,freq,fixfreq,freqerr,amp,fixamp,amperr,phase,fixphase,phaseerr
f0,True,2705.868500,False,0.0178148,0.10263,False,0.000477255,0.290194,False,0.00073837
f1,True,5411.697316,False,0.0455364,0.04016,False,0.000476896,0.604638,False,0.00188829


In [8]:
pyriod.save_solution('SendToKeaton.dat')